In [1]:
%pip install llama-index
%pip install llama-index-graph-stores-neo4j
%pip install llama-index-postprocessor-cohere-rerank
%pip install llmsherpa
%pip install llama-index-embeddings-huggingface


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
  Using cached tokenizers-0.19.1-cp312-cp312-macosx_11_0_arm64.whl.metadata (6.7 kB)
Using cached tokenizers-0.19.1-cp312-cp312-macosx_11_0_arm64.whl (2.4 MB)
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.15.2
    Uninstalling tokenizers-0.15.2:
      Successfully uninstalled tokenizers-0.15.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cohere 5.5.4 requires tokenizers<0.16,>=0.15, but you have tokenizers 0.19.1 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import nest_asyncio

nest_asyncio.apply()

In [3]:
import os

os.environ["OPENAI_API_KEY"] = (
    "sk-proj-"
)

In [4]:
import glob
from llmsherpa.readers import LayoutPDFReader
from llmsherpa.readers.layout_reader import Block
from llama_index.core.schema import BaseNode, Document

pdf_reader = LayoutPDFReader(
    "https://readers.llmsherpa.com/api/document/developer/parseDocument?renderFormat=all"
)

documents = []
for file in glob.glob("./data/pdf" + "/*.pdf"):
    print(file)
    doc = pdf_reader.read_pdf(file)
    block: Block
    for block in doc.chunks():
        metadata = {
            "page": block.page_idx,
            "file_name": os.path.basename(file),
            "tag": block.tag,
        }
        document = Document(text=block.to_context_text(), metadata=metadata)
        documents.append(document)

/Users/holeminhthach/Documents/GitHub/Michael-Exploration/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


./data/pdf/File1.pdf


In [5]:
from llama_index.core.node_parser import SentenceWindowNodeParser

node_parser = SentenceWindowNodeParser.from_defaults(
    window_size=3,
    window_metadata_key="window",
    original_text_metadata_key="original_text",
)
nodes = node_parser.get_nodes_from_documents(documents)

In [5]:
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.huggingface import HuggingFaceEmbedding


llm = OpenAI(model="gpt-3.5-turbo", temperature=0)
# embed_model = OpenAIEmbedding(model_name="text-embedding-3-small")
embed_model = HuggingFaceEmbedding(
    model_name="ls-da3m0ns/bge_large_medical"
)

/Users/holeminhthach/Documents/GitHub/Michael-Exploration/.venv/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


```
docker run \
    -p 7474:7474 -p 7687:7687 \
    -v $PWD/data:/data -v $PWD/plugins:/plugins \
    --name neo4j-apoc \
    -e NEO4J_apoc_export_file_enabled=true \
    -e NEO4J_apoc_import_file_enabled=true \
    -e NEO4J_apoc_import_file_use__neo4j__config=true \
    -e NEO4JLABS_PLUGINS=\[\"apoc\"\] \
    neo4j:latest
```

In [7]:
from llama_index.graph_stores.neo4j import Neo4jPGStore

graph_store = Neo4jPGStore(
    username="neo4j",
    password="llamaindex",
    url="bolt://localhost:7687"
)

In [8]:
from llama_index.core import PropertyGraphIndex

index = PropertyGraphIndex.from_documents(
    documents,
    llm=llm,
    embed_model=embed_model,
    property_graph_store=graph_store,
    show_progress=True,
)

# index = PropertyGraphIndex(
#     nodes=nodes,
#     llm=llm,
#     embed_model=embed_model,
#     property_graph_store=graph_store,
#     show_progress=True,
# )

Generating embeddings: 100%|██████████| 32/32 [00:10<00:00,  3.12it/s]


In [66]:
from llama_index.core.retrievers import (
    CustomPGRetriever,
    VectorContextRetriever,
    TextToCypherRetriever,
)
from llama_index.core.graph_stores import PropertyGraphStore
from llama_index.core.vector_stores.types import VectorStore
from llama_index.core.embeddings import BaseEmbedding
from llama_index.core.prompts import PromptTemplate
from llama_index.core.llms import LLM
from llama_index.postprocessor.cohere_rerank import CohereRerank


from typing import Optional, Any, Union


class MyCustomRetriever(CustomPGRetriever):
    """Custom retriever with cohere reranking."""

    def init(
        self,
        ## vector context retriever params
        embed_model: Optional[BaseEmbedding] = None,
        vector_store: Optional[VectorStore] = None,
        similarity_top_k: int = 4,
        path_depth: int = 1,
        ## text-to-cypher params
        llm: Optional[LLM] = None,
        text_to_cypher_template: Optional[Union[PromptTemplate, str]] = None,
        ## cohere reranker params
        cohere_api_key: Optional[str] = None,
        cohere_top_n: int = 2,
        **kwargs: Any,
    ) -> None:
        """Uses any kwargs passed in from class constructor."""

        self.vector_retriever = VectorContextRetriever(
            self.graph_store,
            include_text=self.include_text,
            embed_model=embed_model,
            vector_store=vector_store,
            similarity_top_k=similarity_top_k,
            path_depth=path_depth,
        )

        self.cypher_retriever = TextToCypherRetriever(
            self.graph_store,
            llm=llm,
            text_to_cypher_template=text_to_cypher_template,
            ## NOTE: you can attach other parameters here if you'd like
        )

        self.reranker = CohereRerank(api_key=cohere_api_key, top_n=cohere_top_n)

    def custom_retrieve(self, query_str: str) -> str:
        """Define custom retriever with reranking.

        Could return `str`, `TextNode`, `NodeWithScore`, or a list of those.
        """
        nodes_1 = self.vector_retriever.retrieve(query_str)
        # nodes_2 = self.cypher_retriever.retrieve(query_str)
        reranked_nodes = self.reranker.postprocess_nodes(
            # nodes_1 + nodes_2, query_str=query_str
            nodes_1, query_str=query_str
        )

        ## TMP: please change
        final_text = "\n\n".join(
            [n.get_content(metadata_mode="llm") for n in reranked_nodes]
            # [n.get_content(metadata_mode="llm") for n in nodes_1]
        )

        return final_text

    # optional async method
    # async def acustom_retrieve(self, query_str: str) -> str:
    #     ...

In [64]:
index.property_graph_store.text_to_cypher_template

'Node properties:\nChunk {embedding: LIST, doc_id: STRING, document_id: STRING, tag: STRING, _node_type: STRING, _node_content: STRING, ref_doc_id: STRING, text: STRING, file_name: STRING, page: INTEGER, id: STRING}\nentity {id: STRING, file_name: STRING, triplet_source_id: STRING, page: INTEGER, tag: STRING, embedding: LIST, name: STRING}\nRelationship properties:\nIs {triplet_source_id: STRING, page: INTEGER, tag: STRING, file_name: STRING}\nSOURCE {triplet_source_id: STRING, page: INTEGER, tag: STRING, file_name: STRING}\n0 {triplet_source_id: STRING, page: INTEGER, tag: STRING, file_name: STRING}\nHas tag {triplet_source_id: STRING, page: INTEGER, tag: STRING, file_name: STRING}\nIn {triplet_source_id: STRING, page: INTEGER, tag: STRING, file_name: STRING}\nIs located in {triplet_source_id: STRING, page: INTEGER, tag: STRING, file_name: STRING}\nIs affiliated with {triplet_source_id: STRING, page: INTEGER, tag: STRING, file_name: STRING}\nNEXT {triplet_source_id: STRING, page: INTE

In [65]:
index.property_graph_store.get_schema_str()

'Node properties:\nChunk {embedding: LIST, doc_id: STRING, document_id: STRING, tag: STRING, _node_type: STRING, _node_content: STRING, ref_doc_id: STRING, text: STRING, file_name: STRING, page: INTEGER, id: STRING}\nentity {id: STRING, file_name: STRING, triplet_source_id: STRING, page: INTEGER, tag: STRING, embedding: LIST, name: STRING}\nRelationship properties:\nIs {triplet_source_id: STRING, page: INTEGER, tag: STRING, file_name: STRING}\nSOURCE {triplet_source_id: STRING, page: INTEGER, tag: STRING, file_name: STRING}\n0 {triplet_source_id: STRING, page: INTEGER, tag: STRING, file_name: STRING}\nHas tag {triplet_source_id: STRING, page: INTEGER, tag: STRING, file_name: STRING}\nIn {triplet_source_id: STRING, page: INTEGER, tag: STRING, file_name: STRING}\nIs located in {triplet_source_id: STRING, page: INTEGER, tag: STRING, file_name: STRING}\nIs affiliated with {triplet_source_id: STRING, page: INTEGER, tag: STRING, file_name: STRING}\nNEXT {triplet_source_id: STRING, page: INTE

In [67]:
# custom_sub_retriever = MyCustomRetriever(
#     index.property_graph_store,
#     include_text=True,
#     vector_store=index.vector_store,

#     cohere_api_key="GHzVxufJiExUdL8hIjonmcApMlKwsZZCOzaKHBfW",
# )

custom_sub_retriever = MyCustomRetriever(
    graph_store=index.property_graph_store,
    include_text=True,
    embed_model=embed_model,
    vector_store=index.vector_store,
    llm=llm,
    text_to_cypher_template=index.property_graph_store.text_to_cypher_template,
    cohere_api_key="",
)

In [68]:
from llama_index.core.query_engine import RetrieverQueryEngine

query_engine = RetrieverQueryEngine.from_args(
    index.as_retriever(sub_retrievers=[custom_sub_retriever]), llm=llm
)

In [56]:
base_retriever = VectorContextRetriever(index.property_graph_store, include_text=True, embed_model=embed_model, vector_store=index.vector_store, similarity_top_k=4, path_depth=1)
base_query_engine = index.as_query_engine(sub_retrievers=[base_retriever])


In [57]:
response = base_query_engine.query(
    "what is the relationship between diabetes and hemophilia?"
)
print(str(response))

Desmopressin is used in the treatment of both diabetes insipidus and hemophilia. It is a drug of choice for substitution treatment of diabetes insipidus and is also used in the treatment of coagulopathies, including hemophilia A and von Willebrand's disease.


In [69]:
response = query_engine.query(
    "what is the relationship between diabetes and hemophilia?"
)

print(str(response))

Desmopressin is used in the management of mild-to-moderate hemophilia A and is also a drug of choice for substitution treatment of diabetes insipidus.


In [70]:
res = index.as_retriever(sub_retrievers=[custom_sub_retriever]).retrieve(
    "what is the relationship between diabetes and hemophilia?"
)
for r in res:
    print(r)

Node ID: dac5fee2-2075-4fcb-b79d-5178a257c82f
Text: page: 4 file_name: File1.pdf tag: table  Here are some facts
extracted from the provided text:  Haemophilia -> Published in -> 2000
Anesthesiol ; :-. () Wieting J.M., Dykstra D.D.,
Ruggiero M.P. et al. : Central nervous  | orrhage after desmopressin
administration. Anesthesiol ; :-. () Wieting J.M.,
Dykstra D.D., R...
Score:  1.000

